In [1]:
import pandas as pd
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset original
df = pd.read_csv("../data/flight_delays_sample.csv") # usa aquí el nombre de tu archivo original

# ============================
# 🧹 LIMPIEZA Y FEATURES
# ============================

# Convertir fechas
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], errors='coerce')

# Crear columnas de fecha
df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek
df['MONTH'] = df['FL_DATE'].dt.month
df['DAY'] = df['FL_DATE'].dt.day

# Columnas horarias
df['DEP_HOUR'] = df['CRS_DEP_TIME'] // 100
df['ARR_HOUR'] = df['CRS_ARR_TIME'] // 100

# Variable objetivo: retrasos mayores a 30 minutos
df['IS_DELAYED'] = (df['ARR_DELAY'] > 30).astype(int)

# Variables temporales adicionales
df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([5, 6]).astype(int)
df['IS_SUMMER'] = df['MONTH'].isin([6, 7, 8]).astype(int)

# Eliminar la columna de fecha para evitar errores con modelos
df = df.drop(columns=['FL_DATE'])

# ============================
# 🧠 ENRIQUECIMIENTO DE DATOS
# ============================

# Retrasos medios por aerolínea, origen y destino
airline_delay = df.groupby('AIRLINE')['ARR_DELAY'].mean()
origin_delay = df.groupby('ORIGIN')['ARR_DELAY'].mean()
dest_delay = df.groupby('DEST')['ARR_DELAY'].mean()

df['AIRLINE_MEAN_DELAY'] = df['AIRLINE'].map(airline_delay)
df['ORIGIN_MEAN_DELAY'] = df['ORIGIN'].map(origin_delay)
df['DEST_MEAN_DELAY'] = df['DEST'].map(dest_delay)

print("✅ Dataset enriquecido con variables de contexto")


✅ Dataset enriquecido con variables de contexto


In [2]:
# Cargar base de datos pública de aeropuertos
airports = pd.read_csv(
    'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',
    header=None,
    names=['id','name','city','country','IATA','ICAO','lat','lon','alt','tz','dst','tz_db','type','source']
)

# Filtrar solo aeropuertos de Estados Unidos
airports_us = airports[(airports['country'] == 'United States') & (airports['IATA'].notnull())]

# Unir coordenadas de origen
df = df.merge(
    airports_us[['IATA', 'lat', 'lon']],
    how='left',
    left_on='ORIGIN',
    right_on='IATA'
)
df.rename(columns={'lat': 'lat_origin', 'lon': 'lon_origin'}, inplace=True)
df.drop(columns=['IATA'], inplace=True)

# Unir coordenadas de destino
df = df.merge(
    airports_us[['IATA', 'lat', 'lon']],
    how='left',
    left_on='DEST',
    right_on='IATA'
)
df.rename(columns={'lat': 'lat_dest', 'lon': 'lon_dest'}, inplace=True)
df.drop(columns=['IATA'], inplace=True)

print("✅ Coordenadas de aeropuertos unidas correctamente.")


✅ Coordenadas de aeropuertos unidas correctamente.


In [3]:
# Crear una muestra (para que no pese demasiado)
df_final = df.sample(300_000, random_state=42)

# Guardar CSV final listo para Tablea
# Ejemplo: si se llama df o data_final, sustitúyelo
df_final.to_csv('flight_delays_tableau.csv', index=False, encoding='utf-8')


print("📁 Archivo guardado como 'flight_delays_tableau.csv'")


📁 Archivo guardado como 'flight_delays_tableau.csv'


In [4]:
df_check = pd.read_csv('flight_delays_tableau.csv')
print(df_check.shape)
df_check.head()


(300000, 46)


,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,CRS_DEP_TIME,...,IS_DELAYED,IS_WEEKEND,IS_SUMMER,AIRLINE_MEAN_DELAY,ORIGIN_MEAN_DELAY,DEST_MEAN_DELAY,lat_origin,lon_origin,lat_dest,lon_dest
0,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,549,PHL,"Philadelphia, PA",ORD,"Chicago, IL",1020,...,1,0,0,5.190780,3.099493,4.703051,39.871899,-75.241096,41.978600,-87.904800
1,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1022,MIA,"Miami, FL",DEN,"Denver, CO",745,...,0,1,1,5.190780,8.729500,4.789569,25.793200,-80.290604,39.861698,-104.672997
2,PSA Airlines Inc.,PSA Airlines Inc.: OH,OH,20397,5223,CHA,"Chattanooga, TN",CLT,"Charlotte, NC",1619,...,0,0,0,4.011259,2.579204,2.211938,35.035301,-85.203796,35.214001,-80.943100
3,PSA Airlines Inc.,PSA Airlines Inc.: OH,OH,20397,5448,PHL,"Philadelphia, PA",IND,"Indianapolis, IN",1400,...,0,0,0,4.011259,3.099493,4.413391,39.871899,-75.241096,39.717300,-86.294403
4,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2518,IND,"Indianapolis, IN",ATL,"Atlanta, GA",1230,...,0,0,1,1.175415,2.942469,0.653679,39.717300,-86.294403,33.636700,-84.428101
